<a href="https://colab.research.google.com/github/faikozcan/MLFraud/blob/main/MLFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Data With  Request To GitHub And Read Columns From CSV

In [9]:
import pandas as pd
import requests
import io
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

url="https://raw.githubusercontent.com/faikozcan/MLFraud/main/dataFiles/Fraud_Data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')),sep = ",",usecols=["user_id","device_id","source","browser","purchase_value","sex","age","class","ip_address"])


In [26]:
descriptiveSet=c[["age", "sex","device_id","ip_address","browser","source"]]
descriptiveSet.describe(include='all')

,age,sex,device_id,ip_address,browser,source
count,151112.000000,151112,151112,1.511120e+05,151112,151112
unique,NaN,2,137956,NaN,5,3
top,NaN,M,NGQCKIADMZORL,NaN,Chrome,SEO
freq,NaN,88293,20,NaN,61432,60615
mean,33.140704,NaN,NaN,2.152145e+09,NaN,NaN
std,8.617733,NaN,NaN,1.248497e+09,NaN,NaN
min,18.000000,NaN,NaN,5.209350e+04,NaN,NaN
25%,27.000000,NaN,NaN,1.085934e+09,NaN,NaN
50%,33.000000,NaN,NaN,2.154770e+09,NaN,NaN
75%,39.000000,NaN,NaN,3.243258e+09,NaN,NaN


Basic one-hot encoding with Pandas

In [ ]:
one_hot_encoded_training_predictors=pd.get_dummies(c)

Split Test And Training Data

In [1]:
y = c['class']
X = c.drop(['class'], axis = 1)
# Split the dataset to trainand test data
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=0)

NameError: ignored

Random Forest Algoritm

In [5]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 50, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6,
              'max_leaf_nodes': None}
RF_model = RandomForestClassifier(**parameters)
RF_model.fit(train_X, train_y)
RF_predictions = RF_model.predict(test_X)
score = accuracy_score(test_y ,RF_predictions)
print(score)


0.9076976017788131
